In [1]:
from managers.history_manager import HistoryManager
from managers.model_manager import ModelManager
from binance.client import Client
from dotenv import load_dotenv
import os

load_dotenv()

client = Client(api_key=os.getenv("BINANCE_API_KEY"), api_secret=os.getenv("BINANCE_API_SECRET"))


hm = HistoryManager(
    client=client,
    start_str="1 Jan, 2023",
    symbol="BTCUSDT",
    interval="1h",
    timelag=16
)

mm = ModelManager(
    predictor_cols=hm.predictor_cols,
    model_name="hgb", # "logreg", "sgdlog", "rf", "hgb", "linsvc"
    class_weight="balanced",
    random_state=42,
)

In [2]:
hm.last_closed_open_time_ms

1756890000000

In [3]:
hm.df_features.describe()

,EMA,MINUSDM,PLUSDM,CLOSE,CLOSEL1,CLOSEL2,PATT_3OUT,PATT_CMB,RSI,MACD,...,BB_MIDDLE,BB_LOWER,BB_WIDTH,OBV,MFI,AD,ADOSC,STOCH_K,STOCH_D,UP_DOWN
count,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,23400.000000,...,23400.000000,23400.000000,23400.000000,2.340000e+04,23400.000000,2.340000e+04,23400.000000,23400.000000,23400.000000,23400.000000
mean,60873.803146,1407.532787,1357.272873,60904.033832,60899.998691,60895.968832,0.030342,0.106154,51.120447,28.112150,...,60873.772545,60096.162903,0.025135,1.518168e+06,50.557484,1.824400e+06,336.875247,52.540046,52.540860,0.509915
std,30369.929840,1205.884169,1048.873781,30379.849480,30379.445301,30379.050957,0.171530,0.308041,11.278140,359.530818,...,30372.091716,30010.166618,0.020450,2.358124e+05,16.865567,4.631339e+05,2524.632575,24.314052,23.604122,0.499912
min,16642.640147,26.292613,44.336893,16622.580000,16622.580000,16622.580000,0.000000,0.000000,6.348371,-2319.577890,...,16633.050625,16532.810810,0.001447,3.020019e+04,3.053455,-1.453758e+04,-49215.947928,1.308173,2.899614,0.000000
25%,29414.799741,555.020223,580.734647,29430.765000,29430.245000,29430.155000,0.000000,0.000000,44.076555,-106.767787,...,29418.084844,29255.013390,0.011785,1.464761e+06,38.207281,1.520061e+06,-292.193390,31.823074,32.411789,0.000000
50%,61201.011907,1072.555965,1100.784921,61169.410000,61168.200000,61167.365000,0.000000,0.000000,50.968904,13.397451,...,61178.564063,60406.187537,0.018978,1.568541e+06,50.298497,2.006103e+06,77.700379,53.155470,52.971346,1.000000
75%,87688.371457,1872.279113,1842.432494,87868.567500,87854.485000,87839.525000,0.000000,0.000000,58.006824,167.940295,...,87802.476875,86341.751718,0.031744,1.659974e+06,62.769590,2.230244e+06,532.389498,73.631490,73.034338,1.000000
max,122508.166175,12901.038723,9949.274388,123847.830000,123847.830000,123847.830000,1.000000,1.000000,90.673497,2390.735922,...,122649.358750,121169.752413,0.200916,1.736033e+06,100.000000,2.308282e+06,46270.662616,99.596606,98.555894,1.000000


In [ ]:
X, y_cls, y_reg = hm.dataset_dual

In [5]:
acc = mm.train(X, y)
print(f"Holdout accuracy: {acc:.3f}")

Holdout accuracy: 0.527


In [6]:
imp = mm.feature_importances()
if imp is not None:
    print(imp.head(10))
